In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

In [18]:
print(tf.__version__)
print(tfp.__version__)

2.0.0
0.8.0


In [3]:
target_dist = tfd.MultivariateNormalDiag(
    loc=tf.cast(tf.zeros([32 * 32 * 1]), tf.float32),
    scale_diag=tf.cast(tf.ones([32 * 32 * 1]), tf.float32)
)

In [4]:
target_dist.sample(1)

<tf.Tensor: id=62, shape=(1, 1024), dtype=float32, numpy=
array([[-0.7871542 , -0.25464413, -0.560056  , ...,  0.25276813,
         0.2677457 , -2.718283  ]], dtype=float32)>

In [6]:
samples = target_dist.sample(256)
prob = target_dist.prob(samples)
print('shape: {}'.format(tf.shape(prob)))
print('mean : {}'.format(tf.reduce_mean(prob)))

shape: [256]
mean : 0.0


In [9]:
print('whole mean {}'.format(tf.reduce_mean(samples)))
print('whole std {}'.format(tf.math.reduce_std(samples)))

whole mean -0.00014054536586627364
whole std 1.0018740892410278


In [13]:
target_dist2 = tfd.Normal(loc=0., scale=1.)
prob = target_dist2.prob(samples)
print('shape: {}'.format(tf.shape(prob)))
print('mean : {}'.format(tf.reduce_mean(target_dist.prob(samples))))

shape: [ 256 1024]
mean : 0.0


In [16]:
print('td log_prob : {}'.format(tf.reduce_mean(target_dist.log_prob(samples))))
print('td2 log_prob : {}'.format(tf.reduce_mean(tf.reduce_sum(target_dist2.log_prob(samples), axis=[-1]))))
# => forget Covariance matrix ???
# => log_prob is too low, I think

td log_prob : -1454.9139404296875
td2 log_prob : -1454.9139404296875


In [26]:
tf.reduce_mean(tf.abs(target_dist.prob(samples)))

<tf.Tensor: id=653, shape=(), dtype=float32, numpy=0.0>

In [33]:
# https://github.com/openai/glow/blob/master/tfops.py#L443-L455
import numpy as np
logsd = tf.zeros(tf.shape(samples)[1:])
mean = tf.zeros(tf.shape(samples)[1:])

def flatten_sum(_logps):
    return tf.reduce_sum(_logps, [1])

logps = lambda x : - 0.5 * (np.log (2 * np.pi) + 2. * logsd + (x - mean) ** 2 / tf.exp(2. * logsd))
logp = lambda x: flatten_sum(logps(x))

print('log_prob (openAI) : {}'.format(tf.reduce_mean(logp(samples))))

log_prob (openAI) : -1454.9139404296875
